In [1]:
# change root path one level up
import os
os.chdir("../acnportal/")


In [3]:
# lightgbm and optuna
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import pytz
import seaborn as sns
from tqdm.notebook import tqdm

CAT_FEATURES = [
    "stationID",
    "spaceID",
    "siteID",
    "clusterID",
    "userID",
    "paymentRequired",
    "connectionTime_Weekday",
    "connectionTime_Hour",
    "connectionTime_Month",
    "connectionTime_is_holiday",
    "connectionTime_is_weekend",
    "connectionTime_weekday_hour",
]


In [4]:
sessions = pd.read_csv("../data/caltech_test_data.csv", index_col=[0, 1])
# drop idle_time column
sessions = sessions.drop(columns=["idle_time"])
sessions


stationID spaceID  siteID  \
connection_time_copy _id                                                     
2018-04-30           5bc915caf9af8b0dad3c0660   2-39-88-24  CA-314       2   
2018-05-07           5bc917d0f9af8b0dc677b8bb  2-39-139-28  CA-303       2   
2018-05-11           5bc9190ff9af8b0dc677b9c1   2-39-88-24  CA-314       2   
2018-05-14           5bc919c3f9af8b0dc677ba32  2-39-79-380  CA-489       2   
                     5bc919c3f9af8b0dc677ba62  2-39-139-28  CA-303       2   
...                                                    ...     ...     ...   
2019-12-30           5e1fbc9ff9af8b5391bcd6fc   2-39-89-25  CA-315       2   
                     5e1fbc9ff9af8b5391bcd700  2-39-79-377  CA-325       2   
2019-12-31           5e210e1ff9af8b57bb4f54fa   2-39-89-25  CA-315       2   
                     5e210e1ff9af8b57bb4f54fd  2-39-79-377  CA-325       2   
                     5e210e1ff9af8b57bb4f5500  2-39-139-28  CA-303       2   

                                               clusterID  kWhDelivered  \
connection_time_copy _id                                                 
2018-04-30           5bc915caf9af8b0dad3c0660         39        47.808   
2018-05-07           5bc917d0f9af8b0dc677b8bb         39        27.683   
2018-05-11           5bc9190ff9af8b0dc677b9c1         39        17.485   
2018-05-14           5bc919c3f9af8b0dc677ba32         39        11.795   
                     5bc919c3f9af8b0dc677ba62         39         3.076   
...                                                  ...           ...   
2019-12-30           5e1fbc9ff9af8b5391bcd6fc         39         5.814   
                     5e1fbc9ff9af8b5391bcd700         39         6.509   
2019-12-31           5e210e1ff9af8b57bb4f54fa         39         4.793   
                     5e210e1ff9af8b57bb4f54fd         39         6.618   
                     5e210e1ff9af8b57bb4f5500         39        15.813   

                                               userID  parking_time  \
connection_time_copy _id                                              
2018-04-30           5bc915caf9af8b0dad3c0660      22      9.335000   
2018-05-07           5bc917d0f9af8b0dc677b8bb      61     10.611944   
2018-05-11           5bc9190ff9af8b0dc677b9c1      22      9.028333   
2018-05-14           5bc919c3f9af8b0dc677ba32      66      9.321944   
                     5bc919c3f9af8b0dc677ba62      61      1.121944   
...                                               ...           ...   
2019-12-30           5e1fbc9ff9af8b5391bcd6fc     743      2.066667   
                     5e1fbc9ff9af8b5391bcd700    1124      7.735556   
2019-12-31           5e210e1ff9af8b57bb4f54fa     743      1.963889   
                     5e210e1ff9af8b57bb4f54fd    1124      7.338333   
                     5e210e1ff9af8b57bb4f5500     838     21.891389   

                                               kWhRequested  \
connection_time_copy _id                                      
2018-04-30           5bc915caf9af8b0dad3c0660          59.5   
2018-05-07           5bc917d0f9af8b0dc677b8bb           8.0   
2018-05-11           5bc9190ff9af8b0dc677b9c1          17.5   
2018-05-14           5bc919c3f9af8b0dc677ba32           8.0   
                     5bc919c3f9af8b0dc677ba62           5.0   
...                                                     ...   
2019-12-30           5e1fbc9ff9af8b5391bcd6fc           8.0   
                     5e1fbc9ff9af8b5391bcd700           8.0   
2019-12-31           5e210e1ff9af8b57bb4f54fa           8.0   
                     5e210e1ff9af8b57bb4f54fd           8.0   
                     5e210e1ff9af8b57bb4f5500          36.0   

                                               Requested_parking_time  \
connection_time_copy _id                                                
2018-04-30           5bc915caf9af8b0dad3c0660                9.166667   
2018-05-07           5bc917d0f9af8b0dc677b8bb                1.000000   
2018-05-11           5bc9190ff9af8b

In [5]:
# testing data is collected from Dec. 1, 2018 to Jan. 1, 2019
test = sessions.loc[(slice("2018-12-01", "2019-01-01"), slice(None)), :]


In [6]:
# evaluate smape
def smape(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred) / np.abs(y_true + y_pred)) * 100


# evaluate smape of requested parking time
print(
    "smape of requested parking time: ",
    smape(test["Requested_parking_time"], test["parking_time"]),
)


smape of requested parking time:  23.0599293684014


In [7]:
# evaluare mae of requested parking time
print(
    "mae of requested parking time: ",
    np.mean(np.abs(test["Requested_parking_time"] - test["parking_time"])),
)


mae of requested parking time:  2.5341190876014554


In [8]:
# evaluate smape of requested energy
print("smape of requested energy: ", smape(test["kWhRequested"], test["kWhDelivered"]))


smape of requested energy:  27.933973390963622


In [9]:
# evaluare mae of requested energy
print(
    "mae of requested energy: ",
    np.mean(np.abs(test["kWhRequested"] - test["kWhDelivered"])),
)


mae of requested energy:  7.1740579345088165


In [10]:
# put all metrics in a dataframe
metrics = pd.DataFrame(
    {
        "smape of requested parking time": [
            smape(test["Requested_parking_time"], test["parking_time"])
        ],
        "mae of requested parking time": [
            np.mean(np.abs(test["Requested_parking_time"] - test["parking_time"]))
        ],
        "smape of requested energy": [
            smape(test["kWhRequested"], test["kWhDelivered"])
        ],
        "mae of requested energy": [
            np.mean(np.abs(test["kWhRequested"] - test["kWhDelivered"]))
        ],
    }
)
# save metrics to csv
metrics.to_csv("./metrics_user_input.csv")
metrics


,smape of requested parking time,mae of requested parking time,smape of requested energy,mae of requested energy
0,23.059929,2.534119,27.933973,7.174058
